# 🔤 Taalverwerking

## Natural Language Processing
(target-nlp)=
_Natural language processing_ (NLP) is, naast _computer vision_, een domein binnen _machine learning_ waar neurale netwerken voor grote doorbraken hebben gezorgd. **NLP richt zich op het automatisch begrijpen, interpreteren en genereren van menselijke taal**. Het is een domein met vele vertakkingen en een lange geschiedenis in _machine learning_. Sinds de introductie van de zogenaamde _Transformer_ architectuur ([](https://doi.org/10.48550/arXiv.1706.03762)) is deze familie van neurale netwerken de dominante modelbenadering geworden in dit domein[^vision_transformer]. Deze architectuur gaf aanleiding tot de huidige golf van _Large Language foundation_ modellen.  

[^vision_transformer]: en andere domeinen zoals beeldherkenning -en generatie.

Net zoals bij beelddata hebben we hier te maken met {ref}`ongestructureerde data <target-unstructured-data>` (Tekstdocumenten: Word-bestanden, PDF’s, enz.).  

:::{important} Eigenschappen
- **Sequentiële patronen**: We zijn op zoek naar betekenisvolle patronen binnen een strikte volgorde van karakters, woorden, zinnen, paragrafen enz.
- **Variabele lengte**: Teksten kunnen sterk variëren in lengte, van enkele woorden tot lange documenten
- **Complexe afhankelijkheidsstructuur**: Betekenis hangt af van context, woordvolgorde en taalkundige nuances. Die afhankelijkheid speelt zich af zowel in de onmiddellijke nabijheid (bv. dezelfde zin) als op soms zeer lange afstanden (bv. conversaties) 
- **Discrete inputs**: Woorden moeten vertaald worden naar numerieke features (zie: _tokenization_)
- **Veel trainingsdata nodig**
- **_Supervised_, _semi-supervised_ of _self-supervised learning_**
- **Vrijwel altijd op basis van neurale netwerk modellen**
:::

Alles draait om de **automatische extractie van semantische patronen en taalkundige structuren**. Vroege NLP toepassingen waren vaak gebaseerd op combinaties van _rule based_ en statistische verwerking. Tot aan de doorbraak van _Transformer_ modellen, deden neurale netwerken eerst hun intrede bij NLP via _recurrent neural networks_ ([_RNN_](https://en.wikipedia.org/wiki/Recurrent_neural_network)), gevolgd door _long short-term memory_ ([_LSTM_](https://en.wikipedia.org/wiki/Long_short-term_memory)) en _gated recurrent unit_ ([_GRU_](https://en.wikipedia.org/wiki/Gated_recurrent_unit)) neurale netwerken.

:::{note} Rule-based systemen
:icon: false
:class: dropdown
Een voorbeeld van een klassieke _rule-based_ benadering is _Part-of-Speech (POS) tagging_ met handgeschreven regels.

[![](https://img.shields.io/badge/Wikipedia-000?logo=wikipedia&logoColor=fff&style=flat)](https://en.wikipedia.org/wiki/Part-of-speech_tagging)

_Part-of-Speech tagging_ is een klassieke NLP taak waarbij elk woord in een zin een grammaticale categorie krijgt toegewezen (zelfstandig naamwoord, werkwoord, bijvoeglijk naamwoord, etc.).

Vroege systemen gebruikten regels voor het Engels zoals:
- Woorden die eindigen op "-ing" zijn waarschijnlijk werkwoorden
- Woorden die volgen op "the" zijn waarschijnlijk zelfstandige naamwoorden
- Woorden in hoofdletters aan het begin van een zin zijn waarschijnlijk eigennamen
- Woorden die eindigen op "-ly" zijn waarschijnlijk bijwoorden
- enz.
:::

## _Transformers_
(target-transformers)=
Moderne NLP gebeurt bijna uitsluitend op basis van **_Transformers_**. Het is een specifieke neurale netwerkarchitectuur die in 2017 werd geïntroduceerd. Hun succes vloeit voort uit het feit dat ze **woorden in hun context begrijpen via zogenaamde _self-attention_**. Dit laatste is een heel krachtig mechanisme om met de complexe afhankelijkheidsstructuren bij taalverwerking om te gaan.

De oorspronkelijke architectuur ziet er als volgt uit (figuur uit [](https://doi.org/10.48550/arXiv.1706.03762)):

[![](../../../img/attention1.png)](https://arxiv.org/pdf/1706.03762)

### _Self-attention_
Door dit mechanisme kan een model bij het verwerken van een bepaald woord naar _alle_ andere woorden in de sequentie "kijken" en "beslissen" welke het belangrijkst zijn voor het begrip van dat specifieke woord. Voor mensen is dit evident, maar het is verre van evident gebleken om dit in _machine learning_ na te bootsen[^earlier].

[^earlier]: Eerdere benaderingen zoals _RNN_, _LSTM_ en _GRU_ neurale netwerken schoten hier telkens in te kort.
  
Neem bijvoorbeeld de zin:
> _Het **_vliegtuig_** kon niet vertrekken omdat **_het_** een technisch probleem had_.  

Als mens begrijpen we onmiddellijk dat **_het_** hier verwijst naar **_vliegtuig_**, maar hoe lost _self-attention_ dit op?  
Om dit te begrijpen, kunnen we denken aan een bibliotheek waarin we ($\approx$ het model) op zoek zijn naar informatie over een bepaald onderwerp ($\approx$ token). Die informatie staat verspreid over verschillende boeken ($\approx$ andere tokens in de sequentie). _Self-attention_ voert een dergelijke zoektocht uit op basis van drie belangrijke componenten die moeten worden geleerd tijdens de training.
1. **_Query_ (Q)**: De _zoekvraag_; dit is bijvoorbeeld (een abstracte vorm van): "Ik ben een voornaamwoord... naar wie verwijs ik?"

2. **_Key_ (K)**: De eigenschappen die een woord/token _adverteert_; bijvoorbeeld: "Ik ben een zelfstandig naamwoord, een mogelijk onderwerp...". Dit kunnen we vergelijken met de labels op de rug van de boeken in de bibliotheek.

3. **_Value_ (V)**: De echte _betekenis_ van het woord in deze context. Denk aan de daadwerkelijke inhoud van een boek.  
  
De verwerking gaat dan als volgt:
1. Voor ieder token worden de Q, K en V vectoren berekend
2. Ieder token vergelijkt zijn _Query_ met de _Keys_ van de andere woorden wat resulteert in een genormaliseerde score. Bijvoorbeeld:
    - `Q_het` • `K_Het` = 0.0 (niet relevant)
    - `Q_het` • `K_vliegtuig` = 0.88 (zeer relevant)
    - ...
    - `Q_het` • `K_probleem` = 0.65
3. Aan de hand van die score weegt ieder woord de _Value_ vectoren (dus de eigenlijke betekenis) van andere woorden.  
  
In de architectuur zitten niet één, maar verschillende (_multi_) _self-attention heads_. Deze worden elk onafhankelijk van elkaar getraind en toegepast. Zo kan bij eenzelfde token tegelijk rekening gehouden worden met verschillende taalkundige eigenschappen (zie onderstaande figuur uit [](https://doi.org/10.48550/arXiv.1706.03762)).

[![](../../../img/attention2.png)](https://arxiv.org/pdf/1706.03762)

De output van de verschillende _heads_ wordt samengevoegd in de output naar volgende lagen. Door hun diepe structuur (zie `N x` bij de blokken in de figuur) kunnen transformers ook veel grotere verbanden leren leggen, dus niet enkel op het niveau van individuele zinnen, maar ook uiteenzettingen, conversaties, enz.

### _Encoder-Decoder_
De oorspronkelijke _Transformer_ architectuur ([](https://doi.org/10.48550/arXiv.1706.03762)) bestaat uit twee hoofdstructuren die samenwerken bij _sequence-to-sequence_ taken zoals _machine translation_:

#### Encoder
(links in de figuur)
- Verwerkt de  volledige input sequentie (bv. een zin in het Frans)
- Bestaat uit verschillende identieke lagen (6 in de originele architectuur)
- Elke laag bevat _multi-head self-attention_ waarbij tokens naar alle andere tokens in de sequentie kunnen "kijken"
- _Output_: Een _dense_ contextuele representatie van de _input_

#### Decoder
(rechts in de figuur)  

:::{note} 🤗 Autoregressie
:icon: false
:class: dropdown
[![](https://huggingface.co/datasets/agents-course/course-images/resolve/main/en/unit1/AutoregressionSchema.gif)](https://huggingface.co/learn/agents-course/en/unit1/what-are-llms)
:::

- Genereert de output sequentie _token voor token_ (bv. de vertaling in het Engels)
- Bij elk nieuw gegenereerd token wordt dit terug als input in de decoder aangeboden (dit heet algemeen _autoregressie_)
- Bestaat ook uit verschillende identieke lagen (6 in de originele architectuur)
- Voor de decoder input wordt met _masked multi-head self-attention_ gewerkt: in tegenstelling tot de encoder, gebeurt de verwerking van tokens enkel door te kijken naar voorgaande tokens in de sequentie

:::{note} 🤗 Decoding
:icon: false
:class: dropdown
<iframe src="https://agents-course-decoding-visualizer.hf.space" frameborder="0" width="850" height="450"></iframe>
:::

:::{note} Moderne varianten
Hoewel de originele architectuur een _encoder-decoder_ was, zijn er inmiddels verschillende varianten:
- **_Encoder-only_** (BERT): Alleen voor begrip/classificatie taken
- **_Decoder-only_** (GPT): Voor tekst generatie
- **_Encoder-decoder_** (T5, BART): Voor _sequence-to-sequence_ taken
:::

### Parameters
(target-transformer-parameters)=
De parameters van _transformer_ modellen zijn enorm talrijk en complex. De belangrijkste zijn:
- _Token embeddings_: Ieder token (ID) is gelinkt aan een _dense_ vector in een _embedding_ matrix. Die matrix wordt getraind.
- _Positional embeddings_: Een van de voordelen van _Transformers_ is dat de verwerking van tokens in parallel kan gebeuren. In tegenstelling tot architecturen die berusten op recurrente verbindingen of convoluties, moet daarom expliciete informatie over de relatieve positie van een token in de sequentie meegegeven worden. Dit kan ook via getrainde parameters gebeuren[^pos_emb]. 

[^pos_emb]: In [](https://doi.org/10.48550/arXiv.1706.03762) werden een vaste sinusoïde encodering gebruikt. 
  
Voor elke _attention head_ in elke _layer_:
- _Query (Q) matrix_: Transformeert _input_ naar _query vector_
- _Key (K) matrix_: Transformeert _input_ naar _key vector_  
- _Value (V) matrix_: Transformeert _input_ naar _value vector_
- _Output projection_: Combineert _outputs_ van _multiple heads_

:::{important} _Large Language Models_ (_LLMs_)
Bedrijven zoals OpenAI zijn de transformer architectuur de laatste jaren enorm gaan opschalen om tot de verbluffende performantie te komen die we vandaag kennen:
- **BERT-base**: ~110 miljoen parameters
- **GPT-2**: ~1.5 miljard parameters  
- **GPT-3**: ~175 miljard parameters
- **GPT-4**: Geschat >1 triljoen parameters
:::

### Features
(target-tokenization)=
We weten intussen dat om machine learning toe te passen alle data naar numerieke waarden omgezet moet worden. In NLP neemt dit proces een speciale vorm aan: **_tokenization_**. 

**_Tokenization_ is de eerste en cruciale stap in NLP waarbij ruwe tekst wordt opgedeeld in kleinere eenheden genaamd _tokens_ - ongeacht het model type**. Deze tokens vormen de elementaire deeltjes die door machine learning modellen verwerkt worden.

Er bestaan weliswaar verschillende tokenization strategieën, elk met hun eigen voor- en nadelen:

#### _Word-level_
Dit is de eenvoudigste benadering waarbij tekst wordt opgesplitst op basis van spaties en interpunctie. Elk uniek woord krijgt een eigen ID. 
```
All models are wrong  
| All | models | are | wrong |  
[2460, 4211, 527, 5076]
``` 
Voordelen:  
- Intuïtief en gemakkelijk te begrijpen
- Behoudt woordgrenzen  
  
Nadelen:  
- Groot vocabularium (honderdduizenden tokens)
- Geen representatie voor _out-of-vocabulary_ woorden
- Morfologische varianten worden als volledig verschillende tokens behandeld  
  
#### _Character-level_
Hier wordt ieder individueel karakter een token.
```
All models are wrong  
| A | l | l | _ | m | o | d | e | l | s | _ | a | r | e | _ | w | r | o | n | g |  
[65, 108, 108, 32, 109, 111, 100, 101, 108, 115, 32, 97, 114, 101, 32, 119, 114, 111, 110, 103]
``` 
Voordelen:  
- Klein vocabularium
- Geen _out-of-vocabulary_ problemen
- Kan morfologische patronen leren  
  
Nadelen:  
- Langere sequenties
- Moeilijker om semantische informatie te leren  

#### _Sub-word_
Hierbij splitst men woorden op in kleinere, betekenisvolle eenheden. Voorbeelden zijn [_Byte-Pair Encoding_ (BPE)](https://en.wikipedia.org/wiki/Byte-pair_encoding) en [_WordPiece_](https://huggingface.co/learn/llm-course/en/chapter6/6).  
```
All unmeaningfulish models are wrong
| All | un | mean | ing | ful | ish | models | are | wrong
[1398, 8362, 3263, 7192, 2118, 2365, 2944, 1132, 2488]
```
Voordelen:
- Balans tussen vocabularium grootte en lengte van sequenties
- Kan omgaan met _out-of-vocabulary_ woorden
- Deelt informatie tussen morfologisch gerelateerde woorden
- Meest gebruikt in moderne _transformers_ (BERT, GPT, T5, enz.)  
  
Nadelen:  
- Minder intuïtief
- Vereist voorafgaande training

:::{note} 🤗 Tokenizer playground
:icon: false
:class: dropdown
<iframe
    src="https://agents-course-the-tokenizer-playground.static.hf.space"
    frameborder="0"
    width="850"
    height="450"
></iframe>
:::

### Leeralgoritme

_Transformers_ worden zoals andere neurale netwerken getraind met varianten van _stochastic gradient descent_ en gebruiken _backpropagation_ om gradiënten te berekenen door het hele netwerk. _Loss_ functies nemen verschillende vormen aan per taak.

### Taken

_Transformers_ worden voor een breed scala aan NLP taken ingezet.

#### Klassificatie
Zowel op het niveau van teksten als individuele tokens bestaan verschillende varianten, bijvoorbeeld:
- _Sentiment_ analyse: Bepalen of een tekst positief, negatief of neutraal is
- _Topic_ classificatie: Indelen in thematische categorieën
- _Spam_ detectie: Onderscheiden van ongewenste berichten
- _Named Entity Recognition (NER)_: bv. Is dit een eigennaam?

:::{note} 🌍
:icon: false
:class: dropdown
Voorbeelden: 
- [RoBERTa](https://huggingface.co/docs/transformers/model_doc/roberta)
- [DistilBERT](https://huggingface.co/docs/transformers/model_doc/distilbert)
:::

#### Automatische vertaling
Het model vertaalt tekst van de ene taal naar de andere. Dit gebeurt aan de hand van een _encoder-decoder_ architectuur.

:::{note} 🌍
:icon: false
:class: dropdown
Voorbeelden: 
- [M2M-100](https://huggingface.co/docs/transformers/model_doc/m2m_100)
- [NLLB](https://huggingface.co/docs/transformers/model_doc/nllb)
:::

#### Vragen beantwoorden
Binnen deze taak is het _chatten_ vandaag de meest gekende toepassing. Er vallen specifieke taken onder zoals het opvolgen van instructies, genereren van redeneringen.
Hiervoor worden verschillende (complexe) subtaken opgezet tijdens het trainen.

:::{note} 🌍
:icon: false
:class: dropdown
Voorbeelden: 
- [GPT-4](https://openai.com/gpt-4)
- [Llama](https://llama.meta.com/)
- [Mistral](https://mistral.ai/)
:::

#### _Representation learning_
_Encoder-only_ modellen genereren _embeddings_ die nuttig zijn voor diverse _downstream_ taken, zoals _similarity search_ en _few-/zero-shot_ klassificatie

:::{note} 🌍
:icon: false
:class: dropdown
Voorbeelden: 
- [BERT](https://huggingface.co/docs/transformers/model_doc/bert)
- [E5](https://huggingface.co/intfloat/e5-large-v2)
:::

### Ervaring
Moderne _transformer_ architecturen (LLMs) worden doorgaans in twee fases getraind:
1. _pre-training_
2. _fine-tuning_  
  
#### _Pre-training_
Tijdens _pre-training_ gebeurt training via **_self-supervised learning_**. Hierbij bestaan er verschillende varianten. De meest eenvoudige zijn:

- _Causal language modeling_: Het model voorspelt het volgende token in een sequentie van tokens en krijgt daarop rechtstreeks feedback.
```
The capital of France is _ | target: Paris
```
- _Masked language modeling_: Het model moet leren om gedeeltelijk gemaskeerde input sequenties te reconstrueren.
```
The _ of France is _ | targets: capital, Paris
```
Het is de bedoeling dat het model een generiek begrip opbouwt van een (of meerdere) taal/talen. Dit vraagt enorme volumes aan data en rekenkracht.
  
#### _Fine-tuning_
Na de _pre-training_ worden modellen gericht naar specifieke taken en objectieven. Hier wordt dan gebruik gemaakt van **_supervised_** _training regimes_.  
In de context van moderne chatbots en assistenten worden speciale _fine-tuning_ technieken ingezet, waaronder:
- _Instruction tuning_: het model wordt _supervised_ getraind op diverse taken geformuleerd als instructies:
```
"Vertaal naar Engels: Hallo wereld"
"Vat samen: [lange tekst]"
"Beantwoord de vraag: Wat is ML?"
```
- _Reinforcement Learning from Human Feedback (RLHF)_:
Tijdens de _pre-training_ fase leren LLMs om taal te produceren die quasi niet te onderscheiden is van natuurlijke taal. Ze zijn daarom niet per se goed in het geven van nuttige/correcte/veilige antwoorden. _RLHF_ werd geïntroduceerd door OpenAI en Google DeepMind ([](  
https://doi.org/10.48550/arXiv.1706.03741)) in de aanloop naar [ChatGPT](https://openai.com/index/chatgpt/). Ze lieten op grote schaal een _pre-trained_ model verschillende antwoorden genereren voor bepaalde vragen en vroegen echte mensen de antwoorden te scoren op wenselijkheid.
```
Question: "How do I make pasta?"

Response A: "Boil water, add salt, cook pasta for 8-10 minutes, drain and serve."
Response B: "Pasta pasta pasta delicious yummy food."

Score Response A > Score Response B
```
Met die informatie werd een apart (_reward_) model getraind om voor iedere output van het taalmodel de "menselijke wenselijkheid" te voorspellen. Daarna werd dit _reward_ model gebruikt om het taalmodel te _fine-tunen_ via _reinforcement learning_ richting meer nuttige/correcte/veilige output.

### Evaluatie

Naast klassieke metrieken (bv. voor klassificatie), bestaan er algemeen in de context van NLP veel specifieke varianten zoals: 

- [Word error rate](https://en.wikipedia.org/wiki/Word_error_rate)
- [BLEU](https://en.wikipedia.org/wiki/BLEU) (Bilingual Evaluation Understudy)
- [ROUGE](https://en.wikipedia.org/wiki/ROUGE_(metric)) (Recall-Oriented Understudy for Gisting Evaluation)
- [METEOR](https://en.wikipedia.org/wiki/METEOR)
- [BERTScore](https://arxiv.org/abs/1904.09675)
- [Perplexity](https://en.wikipedia.org/wiki/Perplexity)
- [Seqeval](https://github.com/chakki-works/seqeval)
- ...

Voor veel NLP taken is automatische evaluatie echter ontoereikend en wordt er geïnvesteerd in menselijke scoring:
- _Fluency_: Is de gegenereerde tekst vloeiend en grammaticaal correct?
- _Coherence_: Is de tekst logisch en samenhangend?
- _Relevance_: Is de output relevant voor de input?
- _Factuality_: Zijn de feiten correct?
- _Safety_: Is de output veilig en ethisch verantwoord?

### Voordelen
- _State-of-the-art_ performantie: _Transformers_ behalen de beste resultaten op vrijwel alle NLP _benchmarks_
- _Transfer learning_: _Pre-trained_ modellen zijn herbruikbaar voor vele taken
- Parallellisatie: Snelle training op moderne hardware (GPUs/TPUs)
- Lange-termijn afhankelijkheden: Kunnen context over lange afstanden modelleren zonder de beperkingen van sequentiële verwerking
- _Multi-task learning_: Eén model kan meerdere taken aan
- _Multi-lingual_: Moderne modellen werken vaak over talen heen

### Nadelen
(target-hallucinations)=
- _Training_: Zeer intensief (duurtijd in de orde van maanden)
- _Inference_: Groot geheugen en rekenkracht nodig voor grote modellen; trage responsen
- Energie & water verbruik: Aanzienlijke milieu-impact van _training_ en _deployment_
- Kosten: _Hardware_ (GPUs) en _cloud computing_ zeer kostelijk
- Interpreteerbaarheid: Complexe, ontransparante architectuur
- _Toxiciteit_: Kunnen ongepaste of schadelijke inhoud genereren - moeilijk om 100% te controleren
- **_Hallucinaties_**: Kunnen **overtuigend feitelijk incorrecte _output_ genereren**
- {ref}`Over fitting <target-over-fitting>`: Kunnen trainingsdata memoriseren en lekken
- _Training window_: Om modellen als kennisbronnen te gebruiken moeten we rekening houden met de _training data cutoff_ datum
- _Context window_: Een _context window_ is het maximale aantal tokens dat het model tegelijk kan "zien". Dit kan 4.000 tokens zijn, 8.000, of zelfs meer dan een miljoen in de nieuwste modellen. Informatie buiten dit venster kan niet in rekening worden gebracht en moet, indien nodig, worden aangereikt via een database van _embeddings_[^rag].

[^rag]: zoals in het zogenaamde _Retrieval Augmented Generation_ (_RAG_) framework